In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from pathlib import Path
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [3]:
torch.manual_seed(42)

In [6]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print('hi')
else:
    device = torch.device("cpu")

hi


In [7]:
device

device(type='mps')

In [8]:
training_data_path = Path("../data/processed/train.csv")
testing_data_path = Path("../data/processed/test.csv")

In [9]:
if training_data_path.exists() and testing_data_path.exists():
    print("Data files found.")
else:
    print("Data files not found. Please ensure the paths are correct.")

Data files not found. Please ensure the paths are correct.


In [10]:
df1=pd.read_csv(training_data_path, on_bad_lines='skip')
df2=pd.read_csv(testing_data_path, on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/train.csv'

In [11]:
main_df=pd.concat([df1,df2], ignore_index=True)

NameError: name 'df1' is not defined

In [12]:
X=main_df.iloc[:,1:]
y=main_df.iloc[:,0]

NameError: name 'main_df' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply transformations

In [17]:
# reshape to 28x28
#  data type uint8 for pil
# make to 3d image 2,28,28
# resize to 3,256,256
# centre crop to 224
# to tensor
# normalize 

In [14]:
from torchvision.transforms import transforms
import torchvision.models as models

vgg16=models.vgg16(pretrained=True)

/Users/divyyadav/Desktop/pytorch/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/divyyadav/Desktop/pytorch/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
for params in vgg16.features.parameters():
    params.requires_grad=False

In [17]:
vgg16.classifier=nn.Sequential(
    nn.Linear(in_features=256, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=10)
)

In [20]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [21]:
transformations=transformations=transforms.Compose([
    
    transforms.Resize(256),
    transforms .CenterCrop(224),
    transforms .ToTensor(),
    transforms .Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
])

In [22]:
class CustomDataset(Dataset):
    
    def __init__(self,features,labels,transformations):

        self.features=features
        self.labels=labels
        self.transformations=transformations
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):

        #reshape to 28x28
        feature=self.features[index].reshape(28,28)

        #change the datatype to uint8
        feature=feature.astype(np.uint8)

        #change the shape to 3d image 
        feature=np.stack([feature]*3, axis=0)

        #conver to image 
        img=Image.fromarray(feature)

        #resize to 3,256,256
        transformed_feature=self.transformations(img)

        #return the transformed feature and label
        return transformed_feature,torch.tensor(self.labels[index],dtype=torch.long)
        

In [25]:
model=vgg16
optimizer=optim.Adam(params=model.parameters(),lr=0.01)
loss=nn.CrossEntropyLoss()